In [1]:
! pip install azure-ai-contentsafety

   ---------------------------------------- 0.0/61.3 kB ? eta -:--:--
   --------------------------------- ------ 51.2/61.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 61.3/61.3 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\wnwanne\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.contentsafety import ContentSafetyClient, BlocklistClient
from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import TextCategory
from azure.ai.contentsafety.models import AddOrUpdateTextBlocklistItemsOptions, TextBlocklistItem
from azure.core.credentials import AzureKeyCredential
from azure.ai.contentsafety.models import TextBlocklist
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions

In [ ]:
endpoint = "https://<resource-name>.cognitiveservices.azure.com/"
credential = AzureKeyCredential("")
content_safety_client = ContentSafetyClient(endpoint, credential)
blocklist_client = BlocklistClient(endpoint, credential)

## Analyze Text

In [6]:
# Create a Content Safety client
client = ContentSafetyClient(endpoint, credential)

# Construct a request
request = AnalyzeTextOptions(text="You are an idiot")

# Analyze text
try:
    response = client.analyze_text(request)
except HttpResponseError as e:
    print("Analyze text failed.")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

if hate_result:
    print(f"Hate severity: {hate_result.severity}")
    print(f"Hate severity: {hate_result}")
if self_harm_result:
    print(f"SelfHarm severity: {self_harm_result.severity}")
if sexual_result:
    print(f"Sexual severity: {sexual_result.severity}")
if violence_result:
    print(f"Violence severity: {violence_result.severity}")

Hate severity: 2
Hate severity: {'category': 'Hate', 'severity': 2}
SelfHarm severity: 0
Sexual severity: 0
Violence severity: 0


## Analyze Text w/ Blocklist

In [10]:
# Create a Blocklist client
blocklist_client = BlocklistClient(endpoint, credential)

blocklist_name = "TestBlocklist"
blocklist_description = "Test blocklist management."

try:
    blocklist = blocklist_client.create_or_update_text_blocklist(
        blocklist_name=blocklist_name,
        options=TextBlocklist(blocklist_name=blocklist_name, description=blocklist_description),
    )
    if blocklist:
        print("\nBlocklist created or updated: ")
        print(f"Name: {blocklist.blocklist_name}, Description: {blocklist.description}")
except HttpResponseError as e:
    print("\nCreate or update text blocklist failed: ")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise


Blocklist created or updated: 
Name: TestBlocklist, Description: Test blocklist management.


In [16]:
# add block items
blocklist_client = BlocklistClient(endpoint, credential)

block_item_text_1 = "kill"
block_item_text_2 = "hate"

block_items = [TextBlocklistItem(text=block_item_text_1), TextBlocklistItem(text=block_item_text_2)]
try:
    result = blocklist_client.add_or_update_blocklist_items(
        blocklist_name=blocklist_name, options=AddOrUpdateTextBlocklistItemsOptions(blocklist_items=block_items)
    )
    for block_item in result.blocklist_items:
        print(
            f"BlockItemId: {block_item.blocklist_item_id}, Text: {block_item.text}, Description: {block_item.description}"
        )
except HttpResponseError as e:
    print("\nAdd block items failed: ")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

BlockItemId: 75b81bdc-a6be-4178-8d6c-889c92523ea7, Text: kill, Description: None
BlockItemId: f1cf06f2-e750-420f-9d46-8dc057b4b9ff, Text: hate, Description: None


In [18]:
blocklist_name = "TestBlocklist"
input_text = "I hate you and I want to kill you."

try:
    # After you edit your blocklist, it usually takes effect in 5 minutes, please wait some time before analyzing with blocklist after editing.
    analysis_result = client.analyze_text(
        AnalyzeTextOptions(text=input_text, blocklist_names=[blocklist_name], halt_on_blocklist_hit=False)
    )
    if analysis_result and analysis_result.blocklists_match:
        print("\nBlocklist match results: ")
        for match_result in analysis_result.blocklists_match:
            print(
                f"BlocklistName: {match_result.blocklist_name}, BlockItemId: {match_result.blocklist_item_id}, "
                f"BlockItemText: {match_result.blocklist_item_text}"
            )
except HttpResponseError as e:
    print("\nAnalyze text failed: ")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise


Blocklist match results: 
BlocklistName: TestBlocklist, BlockItemId: f1cf06f2-e750-420f-9d46-8dc057b4b9ff, BlockItemText: hate
BlocklistName: TestBlocklist, BlockItemId: 75b81bdc-a6be-4178-8d6c-889c92523ea7, BlockItemText: kill
